In [2]:
import pandas as pd
import json
from Config import *
import TextDataFrame as tdf
import utils
import Embedding as eb
import numpy as np
from tqdm.notebook import tqdm
import imp
imp.reload(tdf)
imp.reload(utils)
imp.reload(eb)

<module 'Embedding' from '/home/adrian/Projects/MSCA31008-Data-Mining-Principles/FeatureEngineering/NLP/Embedding.py'>

In [3]:
# Read original json, in chunks of 1000
data_reader = pd.read_json('../data/yelp-dataset/yelp_academic_dataset_review.json', chunksize = 1000, lines = True)

In [4]:
# For each chunk, append review id and text to data list and then remove check
data_list = []
count = 0
for chunk in data_reader:
    count +=1
    print(count)
    data_list.append(chunk[['review_id','text']])
    del chunk

1
2
3
4
5
6
7
8
9


In [5]:
# Concatenate list of data frames into one data frame and remove data_list
data = pd.concat(data_list)
del data_list

In [ ]:
# Initialize dataset preprocessor with default cleaning procedure and apply to texts
df_processor = tdf.DatasetProcessor(DEFAULT_CLEAN_PROCEDURE)
text_series = df_processor.clean_text(data['text'])

# Save as npy file
np.save('all_reviews_cleaned.npy', text_series)
text_series = np.load('all_reviews_embedding.npy', allow_pickle = True)

In [4]:
text_col = 'text'
sample_size = 300000
model_size = 10
utils.set_seed(1997)

print("Sample without replacement：", round((1 - sample_size / len(data)) ** model_size * 100,2), "%")
print("Sample without replacement", round((1 - 1 / len(data)) ** (sample_size * model_size) * 100,2), "%")

Sample without replacement： 73.74 %
Sample without replacement 74.08 %


In [15]:
# Set index
index = data['review_id']

In [16]:
del data

In [22]:
# Convert processed text to date frame
data = text_series.to_frame('text')
data.index = index

In [28]:
# Save to parquet
data.to_parquet('cleaned_text.parquet')

In [5]:
# Test chunk
chunk

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,lWC-xP3rd6obsecCYsGZRg,ak0TdVmGKo4pwqdJSTLwWw,buF9druCkbuXLX526sGELQ,4,3,1,1,Apparently Prides Osteria had a rough summer a...,2014-10-11 03:34:02
1,8bFej1QE5LXp4O05qjGqXA,YoVfDbnISlW0f7abNQACIg,RA4V8pr014UyUbDvI-LW2A,4,1,0,0,This store is pretty good. Not as great as Wal...,2015-07-03 20:38:25
2,NDhkzczKjLshODbqDoNLSg,eC5evKn1TWDyHCyQAwguUw,_sS2LBIGNT5NQb6PD1Vtjw,5,0,0,0,I called WVM on the recommendation of a couple...,2013-05-28 20:38:06
3,T5fAqjjFooT4V0OeZyuk1w,SFQ1jcnGguO0LYWnbbftAA,0AzLzHfOJgL7ROwhdww2ew,2,1,1,1,I've stayed at many Marriott and Renaissance M...,2010-01-08 02:29:15
4,sjm_uUcQVxab_EeLCqsYLg,0kA0PAJ8QFMeveQWHFqz2A,8zehGz9jnxPqXtOc7KaJxA,4,0,0,0,The food is always great here. The service fro...,2011-07-28 18:05:01
...,...,...,...,...,...,...,...,...,...
995,hg4vo-f_0BDSGPzMj-LrdA,HN-2o_s91nULyyIBa7x2Vg,b4PgITSqSWoPhyJ-Mjko7Q,3,0,1,0,"Mojo crepes aka when it was ""blue sky"" back in...",2016-07-25 03:48:17
996,CaoFEIT1oAsOPrfIT0L7mQ,4-zeuXyClhI43r2FleGEdw,xGXzsc-hzam-VArK6eTvtw,5,0,0,0,Divine ambiance. Red walls. Cold ones. Love mu...,2018-02-11 03:29:53
997,_RDqlhTily1mSmltqJAp0A,LK3NnWGLcnxU2i8PCHFaTA,VkV9tfARXGzUjqmRdze_WA,5,4,0,1,Went here for a birthday celebration. Birthda...,2010-03-12 18:41:22
998,EKbOCFXKiR_APXI2mXEsmw,mcPMdBzh8EJWATLvHp5H0A,_rt-Z934kfFzgG19nTrIcQ,5,0,0,0,Great drinks at the tequila bar! Cierra recomm...,2016-05-07 02:12:44
